Установка зависимостей

In [ ]:
%pip install python-docx==1.1.0
%pip install pandas==2.2.1
%pip install nltk==3.8.1
%pip install scikit-learn==1.4.0
%pip install fuzzy-c-means==1.7.2
%pip install spacy==3.7.4
!python -m spacy download ru_core_news_sm

Загрузка данных

In [1]:
import pandas as pd
from docx import Document
import os


def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for paragraph in doc.paragraphs:
        full_text.append(paragraph.text)
    return " ".join(full_text)

df = pd.DataFrame(columns=["doc", "text", "vector"])

dataset_path = "dataset-txt/docx/"
for file_path in os.listdir(dataset_path):
    if (file_path.startswith("~$")):
        continue
    text = read_docx(dataset_path + file_path)
    df.loc[len(df.index)] = [file_path, text, []]

df.head()

,doc,text,vector
0,tz_01.docx,2.2 Техническое задание 2.2.1 Общие сведения п...,[]
1,tz_02.docx,2.2 Техническое задание 2.2.1 Общие сведения П...,[]
2,tz_03.docx,2.2. Техническое задание Общие сведения: В дан...,[]
3,tz_04.docx,Техническое задание 2.2.1 Общие сведения Интер...,[]
4,tz_05.docx,2.2 Техническое задание 2.2.1 Общие сведения. ...,[]


Предобработка текста - удаление стоп-слов и приведение к нижнему регистру

In [ ]:
import nltk

nltk.download("stopwords")
nltk.download("punkt")

test_text = "2.2 К этому времени уже было сформировано множество предпосылок его зарождения: среди философов давно шли споры о природе человека и процессе познания мира, нейрофизиологи и психологи разработали ряд теорий относительно работы человеческого мозга и мышления, экономисты и математики задавались вопросами оптимальных расчётов и представления знаний о мире в формализованном виде; наконец, зародился фундамент математической теории вычислений — теории алгоритмов — и были созданы первые компьютеры."

In [3]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

stop_words = set(stopwords.words("russian"))
punctuation = set(string.punctuation)
print(stop_words)
print(punctuation)


def preprocess_text(text):
    words = word_tokenize(text.lower())
    filtered_words = [
        word for word in words if word not in stop_words and word not in punctuation
    ]
    return " ".join(filtered_words)

print(test_text)
print(preprocess_text(test_text))

{'там', 'сам', 'после', 'нас', 'есть', 'будет', 'когда', 'не', 'ему', 'впрочем', 'он', 'так', 'нее', 'куда', 'уже', 'тем', 'ну', 'два', 'были', 'много', 'опять', 'но', 'что', 'сейчас', 'нельзя', 'ли', 'иногда', 'был', 'с', 'теперь', 'тут', 'совсем', 'тоже', 'разве', 'быть', 'только', 'вот', 'себя', 'надо', 'при', 'другой', 'вы', 'такой', 'да', 'как', 'наконец', 'здесь', 'у', 'тогда', 'была', 'какая', 'свою', 'больше', 'ни', 'ним', 'том', 'этой', 'из', 'один', 'их', 'него', 'может', 'через', 'ее', 'моя', 'она', 'мне', 'о', 'чтобы', 'уж', 'то', 'потому', 'на', 'хорошо', 'они', 'вдруг', 'нет', 'про', 'вам', 'еще', 'этого', 'под', 'чем', 'более', 'зачем', 'во', 'всегда', 'ты', 'этот', 'какой', 'этом', 'никогда', 'лучше', 'эту', 'даже', 'бы', 'об', 'за', 'в', 'ничего', 'тот', 'эти', 'конечно', 'раз', 'себе', 'со', 'нибудь', 'всю', 'мы', 'к', 'кто', 'по', 'чтоб', 'хоть', 'мой', 'можно', 'для', 'его', 'всего', 'все', 'почти', 'ж', 'вас', 'а', 'будто', 'тебя', 'перед', 'если', 'от', 'я', 'них'

In [4]:
df["text"] = df.apply(lambda row: preprocess_text(row["text"]), axis=1)

df.head()

,doc,text,vector
0,tz_01.docx,2.2 техническое задание 2.2.1 общие сведения п...,[]
1,tz_02.docx,2.2 техническое задание 2.2.1 общие сведения п...,[]
2,tz_03.docx,2.2. техническое задание общие сведения данной...,[]
3,tz_04.docx,техническое задание 2.2.1 общие сведения интер...,[]
4,tz_05.docx,2.2 техническое задание 2.2.1 общие сведения 1...,[]


Предобработка текста - удаление спецсимволов

In [5]:
import re

regex = re.compile("[^a-zA-Zа-яА-Я]")

def clean_from_specs(text):
    after_clean_words = [
        token for token in word_tokenize(text) if len(regex.sub("", str(token))) > 2
    ]
    return " ".join(after_clean_words)

print(preprocess_text(test_text))
print(clean_from_specs(preprocess_text(test_text)))

2.2 этому времени сформировано множество предпосылок зарождения среди философов давно шли споры природе человека процессе познания мира нейрофизиологи психологи разработали ряд теорий относительно работы человеческого мозга мышления экономисты математики задавались вопросами оптимальных расчётов представления знаний мире формализованном виде зародился фундамент математической теории вычислений — теории алгоритмов — созданы первые компьютеры
этому времени сформировано множество предпосылок зарождения среди философов давно шли споры природе человека процессе познания мира нейрофизиологи психологи разработали ряд теорий относительно работы человеческого мозга мышления экономисты математики задавались вопросами оптимальных расчётов представления знаний мире формализованном виде зародился фундамент математической теории вычислений теории алгоритмов созданы первые компьютеры


In [6]:
df["text"] = df.apply(lambda row: clean_from_specs(row["text"]), axis=1)

df.head()

,doc,text,vector
0,tz_01.docx,техническое задание общие сведения полное наим...,[]
1,tz_02.docx,техническое задание общие сведения полное наим...,[]
2,tz_03.docx,техническое задание общие сведения данной глав...,[]
3,tz_04.docx,техническое задание общие сведения интернет-ма...,[]
4,tz_05.docx,техническое задание общие сведения полное наим...,[]


Предобработка текста - лемматизация

In [7]:
import spacy

nlp = spacy.load("ru_core_news_sm")


def lemmatize(text):
    lemmatized_words = [
        token.lemma_
        for token in nlp(text)
    ]
    return " ".join(lemmatized_words)

print(clean_from_specs(preprocess_text(test_text)))
print(lemmatize(clean_from_specs(preprocess_text(test_text))))

этому времени сформировано множество предпосылок зарождения среди философов давно шли споры природе человека процессе познания мира нейрофизиологи психологи разработали ряд теорий относительно работы человеческого мозга мышления экономисты математики задавались вопросами оптимальных расчётов представления знаний мире формализованном виде зародился фундамент математической теории вычислений теории алгоритмов созданы первые компьютеры
это время сформировать множество предпосылка зарождение среди философ давно идти спор природа человек процесс познание мир нейрофизиолог психолог разработать ряд теория относительно работа человеческий мозг мышление экономист математика задаваться вопрос оптимальный расчёт представление знание мир формализованный вид зародиться фундамент математический теория вычисление теория алгоритм создать первый компьютер


In [8]:
df["text"] = df.apply(lambda row: lemmatize(row["text"]), axis=1)

df.head()

,doc,text,vector
0,tz_01.docx,технический задание общий сведение полный наим...,[]
1,tz_02.docx,технический задание общий сведение полный наим...,[]
2,tz_03.docx,технический задание общий сведение данной глав...,[]
3,tz_04.docx,технический задание общий сведение интернет - ...,[]
4,tz_05.docx,технический задание общий сведение полный наим...,[]


Векторизация

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from scipy import sparse

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = sparse.csr_matrix(tfidf_vectorizer.fit_transform(df["text"]))
feature_names = tfidf_vectorizer.get_feature_names_out()

df["vector"] = df.apply(lambda row: tfidf_matrix.toarray()[row.name], axis=1)

print(feature_names)
df.head()

['000001' '05' '08080165' ... 'ясность' 'ясный' 'ящик']


,doc,text,vector
0,tz_01.docx,технический задание общий сведение полный наим...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,tz_02.docx,технический задание общий сведение полный наим...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,tz_03.docx,технический задание общий сведение данной глав...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,tz_04.docx,технический задание общий сведение интернет - ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,tz_05.docx,технический задание общий сведение полный наим...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [10]:
def get_terms(doc_index, threshold = 0.0):
    important_words = [
        word
        for word, score in zip(
            feature_names, df.iloc[doc_index]["vector"]
        )
        if score > threshold
    ]
    return ", ".join(important_words)

df["terms"] = df.apply(lambda row: get_terms(row.name, 0.14), axis=1)
df.head()

,doc,text,vector,terms
0,tz_01.docx,технический задание общий сведение полный наим...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","бухгалтерский, должный, пользователь, работа, ..."
1,tz_02.docx,технический задание общий сведение полный наим...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","заказ, консультант, необходимый, продавец, раб..."
2,tz_03.docx,технический задание общий сведение данной глав...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","goodline, должный, клиент, работа, система, тр..."
3,tz_04.docx,технический задание общий сведение интернет - ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","заказ, интернет, магазин, покупатель, сайт, товар"
4,tz_05.docx,технический задание общий сведение полный наим...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","кабинет, конфигурация, медицинский, методическ..."


Кластеризация

In [11]:
from sklearn.cluster import KMeans
import numpy

num_clusters = 2
kmeans = KMeans(n_clusters=num_clusters, random_state=6)
kmeans.fit(tfidf_matrix)

for cluster_id in range(num_clusters):
    cluster_indices = numpy.where(kmeans.labels_ == cluster_id)[0]
    print(f"Кластер {cluster_id + 1} ({len(cluster_indices)}):")
    cluster_docs = [df.iloc[idx]["doc"] for idx in cluster_indices]
    print("; ".join(cluster_docs))
    print("--------")

Кластер 1 (21):
Архитектура, управляемая модель.docx; Введение в проектирование ИС.docx; Встроенные операторы SQL.docx; Методологии разработки программного обеспечения 2.docx; Методологии разработки программного обеспечения.docx; Методы композиции и декомпозиции.docx; Модели представления данных в СУБД.docx; Некоторые особенности проектирования.docx; Непроцедурный доступ к данным.docx; Процедурное расширение языка SQL.docx; Системные объекты базы данных.docx; Технология создания распр ИС.docx; Требования к проекту.docx; Условия целостности БД.docx; Характеристики СУБД.docx; Этапы разработки проекта1.docx; Этапы разработки проекта2.docx; Этапы разработки проекта3.docx; Этапы разработки проекта4.docx; Этапы разработки проекта5.docx; Язык манипуляции данными.docx
--------
Кластер 2 (20):
tz_01.docx; tz_02.docx; tz_03.docx; tz_04.docx; tz_05.docx; tz_06.docx; tz_07.docx; tz_08.docx; tz_09.docx; tz_10.docx; tz_11.docx; tz_12.docx; tz_13.docx; tz_14.docx; tz_15.docx; tz_16.docx; tz_17.docx; 

Нечеткая кластеризация

In [12]:
from fcmeans import FCM

fcm_model = FCM(n_clusters=num_clusters, m=1.2, max_iter=150, error=1e-5, trained=False, n_jobs=1)
fcm_model.fit(tfidf_matrix.toarray())

labels = fcm_model.predict(tfidf_matrix.toarray())
soft_labels = fcm_model.soft_predict(tfidf_matrix.toarray())

for cluster_id in range(num_clusters):
    cluster_indices = numpy.where(labels == cluster_id)[0]
    print(f"Кластер {cluster_id + 1} ({len(cluster_indices)}): ")
    cluster_docs = [f"{df.iloc[idx]["doc"]}[{soft_labels[idx][cluster_id]}]" for idx in cluster_indices]
    print("; ".join(cluster_docs))
    print("--------")

Кластер 1 (22): 
tz_04.docx[0.5166101191802561]; Архитектура, управляемая модель.docx[0.7582125834754992]; Введение в проектирование ИС.docx[0.5850149903384512]; Встроенные операторы SQL.docx[0.7673415504188015]; Методологии разработки программного обеспечения 2.docx[0.6289763499422395]; Методологии разработки программного обеспечения.docx[0.7249251707190414]; Методы композиции и декомпозиции.docx[0.7015593127921799]; Модели представления данных в СУБД.docx[0.8535653976420321]; Некоторые особенности проектирования.docx[0.8969358461350041]; Непроцедурный доступ к данным.docx[0.743764571013637]; Процедурное расширение языка SQL.docx[0.7978352115015387]; Системные объекты базы данных.docx[0.8268402188139414]; Технология создания распр ИС.docx[0.8640491569646779]; Требования к проекту.docx[0.622781259959001]; Условия целостности БД.docx[0.8318622300590391]; Характеристики СУБД.docx[0.844747662718]; Этапы разработки проекта1.docx[0.865309170452283]; Этапы разработки проекта2.docx[0.91157638

Классификация

In [13]:
categories = ["ТЗ", "Статьи"]

class_df = df.filter(["doc", "vector"], axis=1)
class_df["type"] = class_df.apply(lambda row: 0 if str(row["doc"]).startswith("tz_") else 1, axis=1)
class_df.head()

,doc,vector,type
0,tz_01.docx,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
1,tz_02.docx,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
2,tz_03.docx,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
3,tz_04.docx,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
4,tz_05.docx,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

X = [item.tolist() for item in class_df["vector"].to_numpy()]
y = class_df["type"].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)

In [15]:
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy}")

print(y_test)
print(y_pred)

Точность модели: 1.0
[1 1 1 0 0 1 1 0 1]
[1 1 1 0 0 1 1 0 1]
